In [ ]:
!pip install langchain boto3
!pip install transformers
!pip install sentence_transformers

In [ ]:
pip install langchain==0.0.224

In [ ]:
pip install boto3 --upgrade

In [1]:
import boto3
from langchain.retrievers import AmazonKendraRetriever
from typing import List
from typing import Dict
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint, LLMChain
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

from langchain.docstore.document import Document
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [2]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generated_text"]



content_handler = ContentHandler()


In [3]:
from langchain import PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter


llm2 = SagemakerEndpoint(
endpoint_name="hf-llm-falcon-40b-instruct-bf16-2023-06-23-19-34-33-102",
#endpoint_name="hf-llm-falcon-7b-bf16-2023-06-24-20-08-14-262",
#endpoint_name="hf-llm-falcon-40b-bf16-2023-06-24-20-20-44-608",
model_kwargs={
     "parameters" : {"do_sample": False,
    "top_p": 0.9,
    "temperature": 0.1,
    "max_new_tokens": 200
              }},
region_name="us-east-1",
content_handler=content_handler
)

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 4000,
    chunk_overlap  = 100,
    length_function = len,
)

In [4]:
with open("sotu2.txt") as f:
    state_of_the_union = f.read()

docs = text_splitter.create_documents([state_of_the_union])
#print(texts)

In [5]:
from langchain.docstore.document import Document

for x in docs:
    print(x)
    print("---")

page_content="REPORT ON THE STATE OF THE UNION DELIVERED TO A JOINT SESSION OF                    CONGRESS ON FEBRUARY 7, 2023--PM 1  The PRESIDING OFFICER laid before the Senate the following message from the President of the United States which was which was ordered to lie on the table:To the Congress of the United States:  Mr. Speaker. Madam Vice President. Our First Lady and Second Gentleman. Members of Congress and the Cabinet. Leaders of our military. Mr. Chief Justice, Associate Justices, and retired Justices of the Supreme Court. And you, my fellow Americans.  I start tonight by congratulating the members of the 118th Congress and the new Speaker of the House, Kevin McCarthy. Mr. Speaker, I look forward to working together.  I also want to congratulate the new leader of the House Democrats and the first Black House Minority Leader in history, Hakeem Jeffries.  Congratulations to the longest serving Senate leader in history, Mitch McConnell.  And congratulations to Chuck Schumer

In [6]:
map_prompt_template = """Write a short sentence single line summary for following text:


{text}


Summary:"""


reduce_prompt_template = """Write a summary paragraph of the following:


{text}


Summary:"""

In [7]:
from langchain.chains.summarize import load_summarize_chain

MAP_PROMPT = PromptTemplate(template=map_prompt_template, input_variables=["text"])
REDUCE_PROMPT = PromptTemplate(template=reduce_prompt_template, input_variables=["text"])



chain = load_summarize_chain(llm2, chain_type="map_reduce", return_intermediate_steps=True, map_prompt=MAP_PROMPT,  combine_prompt=REDUCE_PROMPT)
#chain.run(docs)
chain({"input_documents": docs}, return_only_outputs=True)

{'intermediate_steps': [' The President of the United States delivers a State of the Union address to a joint session of Congress, highlighting the progress made in the past two years and emphasizing the importance of working together to continue building a stronger and safer America.',
  ' The text is a speech given by President Joe Biden during his State of the Union address. He highlights the achievements of his administration in terms of job creation, inflation reduction, and infrastructure development. He emphasizes the importance of American ingenuity and manufacturing, and the need for bipartisan cooperation to achieve these goals.',
  ' The President is announcing new infrastructure projects and funding for them, including the Brent Spence bridge, which will create jobs and improve transportation. He is also emphasizing the importance of using American-made materials in these projects.'],
 'output_text': '\n\nIn his State of the Union address, President Joe Biden highlights the